In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Hit:13 http://ppa.launchpad.net/u

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-02-14 18:26:25--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.74MB/s    in 0.2s    

2023-02-14 18:26:25 (5.74 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...|2015-01-31 00:08:00|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...| 

In [5]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

960872
960679
960679


# Transform the Data

In [6]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



## Create the "review_id_table".

In [7]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R1004K426LTDKN|   15688375|B002XUL6YY|     852892099|2014-01-22 00:05:00|
|R1006QGKXQ3Q96|   21739087|B005JVP0LE|     559701518|2014-01-16 00:06:00|
|R1017P0ZGRXSR0|    2501262|B0009P66YK|       6220965|2015-01-26 00:02:00|
|R103HXXDQD9W4Y|   16842563|B008MVVA2Q|     849986272|2014-01-30 00:12:00|
|R10472PWW2S2AW|   52130705|B001D9FGAG|     940154187|2009-01-27 00:01:00|
|R104EYLG2ZXXJ1|   41518340|B007I1PLUI|     906701484|2014-01-27 00:11:00|
|R105KF16VGOILR|   48845798|B0041IEATO|     950717807|2011-01-02 00:09:00|
|R106A2ETV5NZNE|   50381801|B000JQSMQO|     963972840|2012-01-16 00:12:00|
|R106UPPPFIIX3P|   42038724|B001CTI5SM|     243892088|2013-01-17 00:08:00|
|R107IKXXNUAZZC|    1594613|B00JMPN3RQ|     849006421|2015-01-11 00:08:00|
|R1081CHIZX3PZW|   459864

## Create the "products" Table

In [8]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df.select(["product_id","product_title"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B002XUL6YY|Rip Curl Men's A1...|
|B005JVP0LE|Casio Men's Sport...|
|B0009P66YK|Invicta Men's 901...|
|B008MVVA2Q|U.S. Polo Assn. C...|
|B001D9FGAG|Invicta Men's 524...|
|B007I1PLUI|Rudiger Men's R20...|
|B0041IEATO|Invicta Women's 0...|
|B000JQSMQO|Fossil Men's Blue...|
|B001CTI5SM|Invicta Men's 505...|
|B00JMPN3RQ|Swiss Legend Men'...|
|B000K6HILI|Invicta Women's 3...|
|B0051UMEHG|Hadley Roma MS834...|
|B004IBT2D8|Geneva Platinum W...|
|B00EAU0IDA|Compact Watch Bra...|
|B000SSPML6|Seiko Men's SGG71...|
|B0046CIYAG|Roberto Bianci Me...|
|B004RQ5A4O|Watch Band Rubber...|
|B006KY5TLK|Timex Women's T5K...|
|B009S4U1F8|Batman Kids' BAT9...|
|B00GG0TSIS|DKNY Women's NY21...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [9]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   45353385|             1|
|    7166154|             1|
|   27053878|             1|
|   33656295|             1|
|   34320055|             1|
|   12611685|             1|
|   26417168|             5|
|   16885399|             3|
|   49090853|             1|
|   48080283|             1|
|   12937506|             5|
|   42209526|             1|
|   36125184|             1|
|   50308356|             1|
|   13273143|             1|
|   30675744|             1|
|   10196801|             3|
|   10748967|             1|
|   44443429|             1|
|   15687327|             2|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [10]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])
vine_df.show() 

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1004K426LTDKN|          2|            0|          1|   N|
|R1006QGKXQ3Q96|          5|            0|          0|   N|
|R1017P0ZGRXSR0|          5|            0|          0|   N|
|R103HXXDQD9W4Y|          5|            0|          0|   N|
|R10472PWW2S2AW|          4|            0|          0|   N|
|R104EYLG2ZXXJ1|          1|            0|          0|   N|
|R105KF16VGOILR|          4|            1|          1|   N|
|R106A2ETV5NZNE|          5|            0|          0|   N|
|R106UPPPFIIX3P|          5|            0|          0|   N|
|R107IKXXNUAZZC|          5|            0|          0|   N|
|R1081CHIZX3PZW|          4|            1|          1|   N|
|R1092B4NE0WNDD|          4|            0|          0|   N|
|R109WVVOK26661|          1|            0|          0|   N|
|R10AQ92VE5HTJ6|          5|            

# Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_df', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products_df', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers_df', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_df', mode=mode, properties=config)